In [1]:
import cv2
import cv2 as cv
from matplotlib import pyplot as plt
import math
from math import atan2, cos, sin, sqrt, pi
import numpy as np

In [2]:
def convertWheelSpeeds(speed_l,speed_r,speed_factor=3,axle_track=94):
    '''
    Function that takes the raw values of each motor angular speed and converts it into
    translation speed (in mm/s) and rotation speed (rad/sec) with a pixel scaling factor.

    Inputs: - speed_l: speed of left motor (no units, from Thymio measurements)
            - speed_r: speed of right motor (no units, from Thymio measurements)
            - speed_factor: 3 (motor_target of 200 sent to Thymio corresponds to a wheel speed of 66.8 mm/s)
            - axle_track: 94mm (distance between both wheels)

    Output: - v  : The current forward velocity of the Thymio (in mm/s)
            - omega : The current angular velocity of the Thymio (in rad/s)(positive direction corresponding to that of the camera frame)
    '''
    # wheel_radius : 22mm
    # axle track : 94mm (distance between the 2 wheels)
    v = 0.5*(speed_l + speed_r) / speed_factor
    omega = (speed_l - speed_r) / (0.5*axle_track*speed_factor)
    
    return v, omega
    

In [3]:
def px2real(x_px, scalingFactor):
    '''
    Function that converts measurement in pixels to measurement in mm.

    Inputs: - x_px : value in pixels
            - scalingFactor : for image size (1920x1080) corresponding to real length and width (800mm x 450mm), the scalingFactor is 2.4 

    Output: - x_real : value in mm
            
    '''
    x_real = x_px / 2.4
    return x_real
    
    

In [4]:
def real2px(x_real, scalingFactor):
    '''
    Function that converts measurement in mm to measurement in pixels.

    Inputs: - x_real : value in mm
            - scalingFactor : for image size (1920x1080) corresponding to real length and width (800mm x 450mm), the scalingFactor is 2.4 

    Output: - x_px : value in pixels
            
    '''
    x_px = x_real * 2.4
    return x_px
    

In [5]:
'''
Extended Kalman Filter class

Code modified from https://github.com/L42Project/Tutoriels/blob/master/Divers/tutoriel36/KalmanFilter.py
Comments describing Kalman Filter taken from https://stackoverflow.com/questions/74318200/how-to-tune-extended-kalman-filter-on-pykalman

Example of instance: KF=KalmanFilter(0.03333, [0, 0, 0, 0, 0]) (dt = 0.033 is the time between each measurement, x_ini = [0, 0, 0, 0, 0] is the initial state)

For update() function : takes as input the measurement vector z
z[0:2] are the measurements from the camera (px, py, orientation)
z[3:4] are the measurements from the Thymio (forward speed v and angular speed omega)

When Thymio position not found by camera -> z becomes 2x1 vector, with z[0:1] being the forward speed v and angular speed omega
'''


class KalmanFilter():
    def __init__(self, dt, x_ini):
        self.dt = dt

        # Initial State Vector
        self.x=np.matrix([[x_ini[0]], [x_ini[1]], [x_ini[2]], [x_ini[3]], [x_ini[4]]])
    

        # Initialize State Transition Matrix (Warning : changes with time -> updateFk())
        self.Fk=np.eye(5)

        # Initialize Measurement matrix H
        # Used to convert the predicted state estimate into predicted sensor measurements at time k.
        # In this case, H will be the identity matrix since the estimated state maps directly to state measurements data
        # H has the same number of rows as sensor measurements and same number of columns as states.
        self.H=np.eye(5)

        
        # State model noise covariance matrix Q
        # When Q is large, the Kalman Filter tracks large changes in
        # the sensor measurements more closely than for smaller Q.
        # Q is a square matrix that has the same number of rows as states.
        self.Q=np.diag([1, 0.1, 0.01, 1.0, 1.0]) 
        

        # Sensor measurement noise covariance matrix R
        # Has the same number of rows and columns as sensor measurements.
        # If we are sure about the measurements, R will be near zero.
        self.coord_var = 0.01
        self.angle_var = 0.0001
        self.speed_var = 6.0
        self.omega_var = 0
        
        self.R=np.diag([self.coord_var, self.coord_var, self.angle_var, self.speed_var, self.omega_var])

        self.P=np.eye(5)
        
    def updateFk(self):
        '''
        Function that updates the state transition matrix Fk
        '''
        
        self.Fk=np.matrix([[1.0, 0, 0, self.dt*math.cos(self.x[2]), 0],
                           [0, 1.0,  0, self.dt*math.sin(self.x[2]), 0],
                           [0, 0, 1.0, 0, self.dt],
                           [0, 0, 0, 1.0, 0],
                           [0, 0, 0, 0, 1.0]])
        self.Fk = self.Fk.astype(float)

    def predict(self):
        # Update Fk
        self.updateFk()
        # Predict the state estimate (A Priori) at time k based on the state estimate at time k-1
        self.x=np.dot(self.Fk, self.x)
        # Predict the state covariance estimate based on the previous covariance and some noise
        self.P=np.dot(np.dot(self.Fk, self.P), self.Fk.T)+self.Q
        return self.x

    def update(self, z, CameraAccessible = True): # z[0:2] corrpesonds to measurement of camera, z[3:4] corresponds to measurements of wheels
        
        if CameraAccessible:
            self.H=np.eye(5)
            
            self.R=np.diag([self.coord_var, self.coord_var, self.angle_var, self.speed_var, self.omega_var])
        else: # CAUTION : IF CAMERAACCESSIBLE = FALSE, Z IS A 2x1 VECTOR (z[0:1] are now measrurements from wheels)
            # Measurement matrix H (now only v and omega can be observed)
            self.H=np.matrix([[0, 0, 0, 1, 0],
                              [0, 0, 0, 0, 1]])
            
            self.R=np.diag([self.speed_var, self.omega_var])
            
            
        # Compute Kalman gain
        S=np.dot(self.H, np.dot(self.P, self.H.T))+self.R
        inv_S = np.linalg.pinv(S.astype(float))
        K=np.dot(np.dot(self.P, self.H.T),inv_S)

        # Correction / innovation
        # Calculate an updated state estimate (A Posteriori) for time k
        self.x=self.x+np.dot(K, (z-np.dot(self.H, self.x)))
        # Update the state covariance estimate for time k
        I=np.eye(self.H.shape[1])
        self.P=(I-(K*self.H))*self.P


        return self.x

In [6]:
def get_thymio_position(img, img_output, min_red = np.array([20, 102, 0]), max_red = np.array([38, 255, 255])):
    
    '''
    Function that outputs Thymio position, radius and orientation from camera frames

    Inputs: - img : image from camera in BGR format
            - img_output : copy of image img on which will be drawn Thymio's contours
            - min_red : minimum values for Hue, Saturation and Value for hsv mask
            - max_red : minimum values for Hue, Saturation and Value for hsv mask

    Output: - center : tuple (x,y) representing the center of the Thymio (in pixels)
            - radius : radius of the Thymio (in pixels)
            - angle : orientation of the Thymio (in radians)
            - thymio_detected : boolean set to True if Thymio was detected, False if not
    '''
    
    img_hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    img_hsv_blur = cv2.medianBlur(img_hsv, 15)
    mask = cv2.inRange(img_hsv_blur, min_red, max_red)
    mask = cv2.erode(mask, None, iterations=2) # 3x3 kernel used
    mask = cv2.dilate(mask, None, iterations=2)    
    
    contours, hierarchy = cv2.findContours(mask, cv2.RETR_CCOMP, cv2.CHAIN_APPROX_SIMPLE)
    contours = sorted(contours, key=lambda x:cv2.contourArea(x), reverse=True)
    
    cv2.drawContours(img_output, contours, -1, (0,255,0), 3)
    
    thymio_detected = False
    center = (0,0)
    radius = 0
    angle = 0
    
    if len(contours) != 0:

        # find the biggest countour c by the area
        c = max(contours, key = cv2.contourArea)
        if cv2.contourArea(c) > 1000:

            #Minimum Enclosing Circle :
            (x_circle,y_circle),radius = cv2.minEnclosingCircle(c)
            center_circle = (int(x_circle),int(y_circle))
            radius = int(radius)
 
            rect = cv2.minAreaRect(c)
            box = cv2.boxPoints(rect)
            box = np.int0(box)
            cv2.drawContours(img_output,[box],0,(255,255,0),10)
            center_rect = (int(rect[0][0]),int(rect[0][1]))
            

            for contour_idx, contour in enumerate(contours):
                area = cv2.contourArea(contour)
                if area > cv2.contourArea(c)/15 and area < cv2.contourArea(c)/8 and hierarchy[0,contour_idx,3] == 0:
                    thymio_detected = True
                    (x_back,y_back),radius_back = cv2.minEnclosingCircle(contour)
                    center = (int(x_back),int(y_back))
                    radius_back = int(radius_back)
                    cv2.circle(img_output,center,radius_back,(255,0,0),10)
                    
                    angle = math.atan2(center_rect[1]-center[1],center_rect[0]-center[0])

        
    return center, radius, angle, thymio_detected

In [ ]:
'''
Testing Kalman Filter
'''

global center, radius, angle, thymio_detected

forward_speed = 300
angular_speed = 0

KF=KalmanFilter(0.033, [500, 300, math.pi/4, forward_speed, angular_speed])



### INITIALIZE CAMERA
cap = cv2.VideoCapture(0)
fps = int(cap.get(5))
print("fps:", fps)



### MAIN LOOP
count = 0

# Check if the webcam is opened correctly
if not cap.isOpened():
    raise IOError("Cannot open webcam")

# running infinite while loop so that
# program keep running until we close it
while True:
    success, img = cap.read()
    imgResult = img.copy()
    
    thymioLoc, thymioRadius, thymioAngle, thymioDetected = get_thymio_position(img, imgResult)
    
    # KALMAN -------------------------------------------------------------------
    KF.predict()
    if thymioDetected:
        z = np.array([thymioLoc[0],thymioLoc[1],thymioAngle, forward_speed, angular_speed])
        z = np.expand_dims(z, axis=-1)
    else:
        z = np.array([forward_speed, angular_speed]) 
        z = np.expand_dims(z, axis=-1)
    KF.update(z,thymioDetected)
    cv2.circle(imgResult, (int(KF.x[0]), int(KF.x[1])), 10, (255,0,255), 10)
    # KALMAN ----------------------------------------------------------------------
    
    count = count + 1
    print(count)
    print('Thymio center:',thymioLoc, '  Angle:',thymioAngle, '  Thymio Detected:' ,thymioDetected)
    print('Kalman center:',KF.x[0:1],'  Kalman angle:',KF.x[0:1], 'Kalman speed and omega', KF.x[3:4])
 

    # displaying output on Screen
    cv2.imshow("Result", imgResult)
    
    c = cv2.waitKey(1)
    if c == 27:
        break

fps: 30
1
Thymio center: (0, 0)   Angle: 0   Thymio Detected: False
Kalman center: [[507.00035713]]   Kalman angle: [[507.00035713]] Kalman speed and omega [[300.]]
2
Thymio center: (0, 0)   Angle: 0   Thymio Detected: False
Kalman center: [[514.00071427]]   Kalman angle: [[514.00071427]] Kalman speed and omega [[300.]]
3
Thymio center: (0, 0)   Angle: 0   Thymio Detected: False
Kalman center: [[521.0010714]]   Kalman angle: [[521.0010714]] Kalman speed and omega [[300.]]
4
Thymio center: (0, 0)   Angle: 0   Thymio Detected: False
Kalman center: [[528.00142853]]   Kalman angle: [[528.00142853]] Kalman speed and omega [[300.]]
5
Thymio center: (0, 0)   Angle: 0   Thymio Detected: False
Kalman center: [[535.00178567]]   Kalman angle: [[535.00178567]] Kalman speed and omega [[300.]]
6
Thymio center: (0, 0)   Angle: 0   Thymio Detected: False
Kalman center: [[542.0021428]]   Kalman angle: [[542.0021428]] Kalman speed and omega [[300.]]
7
Thymio center: (0, 0)   Angle: 0   Thymio Detected: 